In [3]:
import onnxruntime as ort
import torch
import time

def test_time(path,inputs):

    providers = [("CPUExecutionProvider")]
    sess_options = ort.SessionOptions()
    sess = ort.InferenceSession(path, sess_options=sess_options, providers=providers)
    t0 = time.time()
    for x in range(3):
      outputs2 = sess.run(None,inputs )
    t1 = time.time()
    print("total time 1 inference on CPU: ",path,(t1-t0)/3)

    return outputs2


In [4]:
import torch.jit

model = torch.load("model.pt")
dummy_image0 = torch.randn(1, 1, 800, 800, device="cpu")
dummy_image1 = torch.randn(1, 1, 800,800, device="cpu")
model(dummy_image0,dummy_image1)
print(0)
model = torch.jit.load("script_model.pt")
model(dummy_image0,dummy_image1)
print(1)
model = torch.jit.load("trace_model.pt")
model(dummy_image0,dummy_image1)
print(2)
inputs= {'image0': dummy_image0,'image1': dummy_image1}

test_time("model.onnx",inputs)
print(3)
test_time("model_simplified.onnx",inputs)
print(4)

torch.Size([1, 2046, 5, 5])
0
[1, 2046, 5, 5]
1


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/TopicFM/src/models/topic_fm.py", line 45, in forward
    _17 = ops.prim.NumToTensor(torch.size(feat_c1, 1))
    feat1 = torch.reshape(_14, [_16, -1, int(_17)])
    _18 = (loftr_coarse).forward(feat0, feat1, )
           ~~~~~~~~~~~~~~~~~~~~~ <--- HERE
    _19 = (coarse_matching).forward(_18, _4, _5, _7, _8, _3, other, _6, other0, _0, )
    _20, _21, _22, _23, _24, _25, _26, = _19
  File "code/__torch__/TopicFM/src/models/modules/transformer.py", line 108, in forward
    _53 = torch.slice(resized_feat, 1, 0, 9223372036854775807)
    _54 = annotate(List[Optional[Tensor]], [new_mask])
    _55 = torch.index_put_(_53, _54, _52)
          ~~~~~~~~~~~~~~~~ <--- HERE
    len_topic0 = torch.to(torch.sum(topick1, [-1]), 3)
    selected_ids0 = torch.to(topick1, 11)

Traceback of TorchScript, original code (most recent call last):
/h